# Whole brain data processing

In [1]:
import numpy as np
import pandas as pd
import os, sys
from glob import glob
from h5py import File
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import dask.array as da
from cellProcessing import *
import time
import shutil
import dask

%matplotlib inline
%load_ext autoreload
%autoreload 2

## Set up environments

* Install `fish_proc` from github https://github.com/zqwei/fish_processing

In [14]:
## STEP 1: specify data and save path
dir_root = '/groups/ahrens/ahrenslab/jing/giving_up/20190219/fish2/7dpf-huc_gcamp7ff_gfap_rgeco-GU_slow_fwd-fish02-exp02_20190219_174013/im'
save_root = '/nrs/ahrens/jing/20190219/fish2/7dpf-huc_gcamp7ff_gfap_rgeco-GU_slow_fwd-fish02-exp02_20190219_174013/processed'

## Preproecssing
This will generate the follow files at `save_root` folder
* `motion_fix_.h5` -- reference image
* `trans_affs.npy` -- affine transform
* `Y_2dnorm_ave.h5` -- average image after detrend
* `local_pca_data.zarr` -- local pca denoised image (This might be used a x4 file size comparing to the raw data), which can be removed afterwards

In [16]:
%%time
baseline_percentile = 20  
baseline_window = 1000   # number of frames
numCores = 200  # requests for dask workers
preprocessing(dir_root, save_root, numCores=numCores, window=baseline_window, percentile=baseline_percentile)

<Client: scheduler='tcp://10.36.111.13:38798' processes=150 cores=150>


distributed.comm.tcp - WARNING - Closing dangling stream in <TCP local=tcp://10.36.111.13:36906 remote=tcp://10.36.111.13:44549>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP local=tcp://10.36.111.13:36910 remote=tcp://10.36.111.13:44549>
Worker tcp://10.36.60.22:41815 restart in Job 56019441. This can be due to memory issue.
Worker tcp://10.36.60.37:42837 restart in Job 56019419. This can be due to memory issue.
Worker tcp://10.36.60.21:38928 restart in Job 56019446. This can be due to memory issue.
Worker tcp://10.36.60.30:35957 restart in Job 56019491. This can be due to memory issue.
Worker tcp://10.36.60.21:42774 restart in Job 56019579. This can be due to memory issue.
Worker tcp://10.36.60.11:45372 restart in Job 56019424. This can be due to memory issue.
Worker tcp://10.36.60.21:38450 restart in Job 56019611. This can be due to memory issue.
Worker tcp://10.36.60.34:36449 restart in Job 56019626. This can be due to memory issue.
Worker tcp://10.36.60.18:34944

KilledWorker: ("('array-stack-859a9e523b3f1697372d7ebc8a2359ab', 5259, 0, 0, 0)", 'tcp://10.36.60.21:37703')

Worker tcp://10.36.60.26:42341 restart in Job 56019470. This can be due to memory issue.
Worker tcp://10.36.60.26:43647 restart in Job 56019501. This can be due to memory issue.
Worker tcp://10.36.60.21:37747 restart in Job 56019590. This can be due to memory issue.
Worker tcp://10.36.60.21:37196 restart in Job 56019593. This can be due to memory issue.
Worker tcp://10.36.60.33:45257 restart in Job 56019540. This can be due to memory issue.
Worker tcp://10.36.60.12:37429 restart in Job 56019609. This can be due to memory issue.
Worker tcp://10.36.60.15:36792 restart in Job 56019507. This can be due to memory issue.
Worker tcp://10.36.60.18:42801 restart in Job 56019563. This can be due to memory issue.
Worker tcp://10.36.60.23:40440 restart in Job 56019473. This can be due to memory issue.
Worker tcp://10.36.60.15:41116 restart in Job 56019561. This can be due to memory issue.


## Local PCA
* request a single machine with 32 or 48 cores

In [ ]:
local_pca(save_root)

## Mask
Making a mask for data to decrease the number of computation in demix
* remove low intensity pixel
* remove low snr pixel
* remove low local pixel correlation pixels

This will generate the follow files at `save_root` folder
* `mask_map.h5` -- mask
* `local_correlation_map.h5` -- local correlation
* `masked_local_pca_data.zarr` -- df/f computation
* `masked_downsampled_local_pca_data.zarr` -- downsampled in time -> used for cell segmentation, which can be removed afterwards

In [ ]:
## mask parameters
intensity_percentile = 40  ## throws out low intensity pixels to create mask: try 20-40
dt = 5  # time downsample for cell segmentation
mask_brain(save_root, percentile=intensity_percentile, dt=dt, numCores=20, is_skip_snr=True, save_masked_data=True)

### Check mask

In [ ]:
## Check if mask doesn't look good, choose another value
# percentile is applied on a plane by plane basis (i.e. each plane has the different thresholds)
mask = File(f'{save_root}/mask_map.h5', 'r')['default'].value

In [ ]:
for n_ave_ in mask.squeeze():
    plt.imshow(n_ave_)
    plt.show()

## Demix
This will generate the follow files at `save_root` folder
* `demix_rlt` for each block

In [ ]:
nsplit = 8 # number of split on x and y -- about 256x256 is good
demix_cells(save_root, nsplit = nsplit, numCores = 200)

### Check demix results

In [ ]:
block_id = (0, 2, 0, 0)
check_demix_cells(save_root, block_id, nsplit=8, plot_global=True)

## Compute df/f
One of three choices
* df is from NMF components (where background is not included) -- preferred
* df is on raw data based on the recomputation of cell F using NMF weights
* df/f on pixels without using NMF weights

### df/f on pixels

In [ ]:
baseline_percentile = 20
baseline_window = 1000
numCores = 200
dff = compute_cell_dff_pixels(dir_root, save_root, 
                              numCores=numCores, 
                              window=baseline_window, 
                              percentile=baseline_percentile)

### df/f on raw cell data

In [ ]:
compute_cell_dff_raw(dir_root, save_root, 
                     numCores=numCores, 
                     window=baseline_window, 
                     percentile=baseline_percentile, 
                     nsplit=nsplit)

### df/f on denoised cell data

In [ ]:
compute_cell_dff_NMF(dir_root, save_root, 
                     numCores=numCores, 
                     window=baseline_window, 
                     percentile=baseline_percentile, 
                     nsplit=nsplit, dt=dt)